<a href="https://colab.research.google.com/github/TomoharuKurosu/TomoharuKurosu/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['Capital Share'] = 1 - data['labsh']  # Capital share
data['A'] = data['y_pc'] - data['k_pc']*data['Capital Share']
data['TFP Share'] = data['labsh']
# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['Growth Rate'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['Capital Deepening'] = (grouped_data['k_pc'].diff() * 100)  
data['TFP Growth'] = (grouped_data['A'].diff() * 100)  


# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'Growth Rate': 'mean',
                                            'TFP Growth': 'mean',
                                           'Capital Deepening': 'mean',
                                           'TFP Share': 'mean',
                                       'Capital Share': "mean"}
                                       )

# Print output
print(summary)

             Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
countrycode                                                          
CAN             0.887700    0.648264           1.835712   0.653810   
DEU             0.661055    0.556886           1.198290   0.629270   
FRA             0.829236    0.207893           1.491602   0.618806   
GBR             1.015544   -0.267172           1.379846   0.587007   
ITA            -0.037181   -0.520106           1.103655   0.515165   
JPN             0.710956    0.725321           1.506205   0.569399   
USA             1.536985    0.961097           2.014710   0.606740   

             Capital Share  
countrycode                 
CAN               0.346190  
DEU               0.370730  
FRA               0.381194  
GBR               0.412993  
ITA               0.484835  
JPN               0.430601  
USA               0.393260  
